# Progetto d'esame di Data Analysis in Experimental Physics with Machine Learning
Gruppo composto dagli studenti Luca Attinà, Sharis Feriotto e Matteo Marchisio Caprioglio

Dataset ipotesi: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

Questo dataset non va bene perchè ha fatto data aug sul validation dataset, fallback al plant village originale: https://www.tensorflow.org/datasets/catalog/plant_village

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
import os
import pandas as pd

# from tqdm.notebook import tqdm


In [ ]:
print(tf.config.list_physical_devices(''))


In [ ]:
# Load the PlantVillage dataset from TFDS instead of the new dataset (it performed data aug on the validation set, which is wrong)
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:95%]', 'train[95%:]'],
    shuffle_files=True,
    as_supervised=True,  # returns (image, label) pairs
    with_info=True
)


In [ ]:
# Show example from the dataset
tfds.show_examples(ds_train, ds_info)


In [ ]:
class_names = ds_info.features['label'].num_classes
print(f"Number of classes: {class_names}")
class_names = ds_info.features['label'].names
print(f"Class names: {class_names}")


In [ ]:
ds_info


In [ ]:
# Convert the training dataset to a DataFrame
df = tfds.as_dataframe(ds_train, ds_info)

# Count the number of samples per class
class_counts = df['label'].value_counts().sort_index()

# Print the counts with class names
for idx, count in class_counts.items():
    print(f"{class_names[idx]}: {count} images")


In [ ]:
# useful constants
IMG_SIZE = (128, 128)
BATCH_SIZE = 128 
APPLY_DATA_AUGMENTATION = False
N_EPOCHS = 30
NUM_CLASSES = ds_info.features['label'].num_classes
DROP_RATE = 0.3


In [ ]:
# data_augmentation = tf.keras.Sequential([
    # tf.keras.layers.RandomFlip("horizontal"),
    # tf.keras.layers.RandomFlip("vertical"),
    # tf.keras.layers.RandomRotation(0.1),
    # tf.keras.layers.RandomZoom(0.1),
    # tf.keras.layers.RandomContrast(0.1),
# ])

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, tf.one_hot(label, ds_info.features['label'].num_classes)

# def preprocess_with_aug(image, label):
    # image = tf.image.resize(image, IMG_SIZE)
    # image = data_augmentation(image)  # <-- augment here
    # image = tf.cast(image, tf.float32) / 255.0
    # return image, tf.one_hot(label, ds_info.features['label'].num_classes)



In [ ]:
# Preprocess and batch the datasets
# if APPLY_DATA_AUGMENTATION:
    # print("Data augmentation is enabled.")
    # train_ds = ds_train.map(preprocess_with_aug, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)  # training dataset with data augmentation
# else:
    # print("Data augmentation is disabled.")
    # train_ds = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)  # training dataset without data augmentation

train_ds = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = ds_val.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Activation, BatchNormalization, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, GlobalAveragePooling2D, ReLU, Rescaling
from keras.optimizers.legacy import Adam, SGD
from keras.losses import CategoricalCrossentropy
from keras import regularizers

from keras.metrics import CategoricalAccuracy, Precision, Recall
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [ ]:
def simple_cnn(#input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
                num_classes=NUM_CLASSES):
    model = Sequential([
        # Rescaling(1./255, input_shape=input_shape),
        
        Conv2D(16, (5, 5), padding='same', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(DROP_RATE),
        
        Conv2D(32, (5, 5), padding='same', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(DROP_RATE),
        
        Flatten(),
        
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
def simple_cnn_v2(#input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
                num_classes=NUM_CLASSES):
    model = Sequential([
        # Rescaling(1./255, input_shape=input_shape),
        
        Conv2D(16, (5, 5), padding='same', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(DROP_RATE),
        
        Conv2D(32, (5, 5), padding='same', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(DROP_RATE),

        Conv2D(64, (5, 5), padding='same', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(DROP_RATE),

        # Flatten(),
        GlobalAveragePooling2D(),
        
        Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        # Dropout(DROP_RATE),
        Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
model = simple_cnn_v2()
model.build(input_shape=(None, IMG_SIZE[0], IMG_SIZE[1], 3))  # Build the model with dynamic batch size
model.summary()


In [ ]:
optimizer = Adam(learning_rate=0.0002)
# optimizer = SGD(learning_rate=0.05, momentum=0.9)
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

check_point_filename = 'best_model.h5'  # Default checkpoint filename for now


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get all labels from the training set
labels = []
for _, label in ds_train:
    labels.append(label.numpy())
labels = np.array(labels)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights_dict = dict(enumerate(class_weights))
print(class_weights_dict)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
        ModelCheckpoint(check_point_filename, monitor='val_loss', save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
    ],
    class_weight=class_weights_dict,
)


In [ ]:
# plot training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()
